In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.common.keys import Keys
import wget
from tqdm import tqdm_notebook
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
data_old = pd.read_csv('C:/Users/mdyakova/venv/lncRNA/lncrna/lncbase_data.tar/lncbase_data/lncbase_data/LncBasev2_download.csv', sep='\t')

In [3]:
data_old = data_old[(data_old['species']=='Homo sapiens') | (data_old['species']=='Homo Sapiens')]

In [4]:
data_old.head()

,geneId,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,condition
0,ENSG00000002079,MYH16,hsa-miR-4786-3p,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,treatment: emetine
1,ENSG00000067601,PMS2P4,hsa-miR-24-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN
2,ENSG00000073905,VDAC1P1,hsa-miR-1179,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,no treatment (control)
3,ENSG00000073905,VDAC1P1,hsa-miR-139-5p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN
4,ENSG00000073905,VDAC1P1,hsa-miR-27a-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN


In [5]:
data = data_old.groupby(by=['geneName', 'mirna'], as_index=False).max()

In [8]:
data_loaded = pd.read_csv('diana_data_ver2.csv', header=None, names=('mirna', 'lncrna', 'text'))

In [11]:
data = pd.merge(data, data_loaded, left_on=['geneId', 'mirna'], right_on=['lncrna', 'mirna'], how='left')

In [14]:
data = data[pd.isna(data['text'])]

In [15]:
def request(mirna, lncrna):
    string = ('http://carolina.imis.athena-innovation.gr/diana_tools/web/index.php?r=lncbasev2%2Findex-experimental&miRNAs%5B%5D=&miRNAs%5B%5D='
                        + mirna + '&lncRNAs%5B%5D='+ lncrna + '&filters=0')
    return string

In [16]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"
options = webdriver.ChromeOptions()
#         options.set_headless(True) 
browser = webdriver.Chrome(executable_path='chromedriver.exe', desired_capabilities=caps, options=options)

In [ ]:
for i in tqdm_notebook(range(0, len(data))):
    try:
        example = data.iloc[i]
        mirna = example['mirna']
        lncrna = example['geneId']
        string = request(mirna, lncrna)

        browser.get(string)
        time.sleep(5)

        results = browser.find_elements_by_class_name('button_box')[0]
        results.click()

        start_scroll = 400
        browser.execute_script("window.scrollTo(0, " + str(start_scroll) +")") 

        results2 = browser.find_elements_by_class_name('button_box')[1]
        results2.click()

        result3 = browser.find_elements_by_id('lnc_experimental_results')[0]

        text = result3.text

        pd.DataFrame([[mirna, lncrna, text]]).to_csv('diana_data_ver2.csv', index=None, header=None, mode='a')
    except:
        pass
    
    
    
    
#     break

c:\users\mdyakova\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.
